In [1]:
from os import listdir, path
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

C:\Users\16678039\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<h2><center> Подготовка данных</center></h2>

In [2]:
SOURCE_PATH = './data/sources/'

'''
sources_ws - split by white spaces and \n
sources_ch - split by chars
'''
sources_ws = []
sources_ch = []
for file_name in tqdm(listdir(SOURCE_PATH)):
    with open(path.join(SOURCE_PATH, file_name), 'r', encoding="ansi") as file:
        data=file.read()
        sources_ws.append(data.split())
        sources_ch.append([ord(char) for char in list(data)])

100%|██████████████████████████████████| 21941/21941 [00:02<00:00, 7483.75it/s]


In [3]:
print(len(sources_ws))
print(len(sources_ch))

21941
21941


In [4]:
lengths_ws = [len(data) for data in sources_ws]
lengths_ch = [len(data) for data in sources_ch]
max_l_ws = max(lengths_ws)
min_l_ws = min(lengths_ws)
max_l_ch = max(lengths_ch)
min_l_ch = min(lengths_ch)
print('Максимальная длина разбинения по пробелам: ', max_l_ws)
print('Минимальная длина разбинения по пробелам: ', min_l_ws)
print('Максимальная длина разбинения по символам: ', max_l_ch)
print('Минимальная длина разбинения по символам: ', min_l_ch)

Максимальная длина разбинения по пробелам:  289
Минимальная длина разбинения по пробелам:  1
Максимальная длина разбинения по символам:  1023
Минимальная длина разбинения по символам:  7


In [9]:
PAD_SYMBOL = ord(' ')
flat_sources_ch = [ch for source in sources_ch for ch in source]
flat_sources_ch.append(PAD_SYMBOL)
unique_ch = set(flat_sources_ch)
ch2index = {ch: i for (i, ch) in enumerate(unique_ch)}
index2ch = {i: ch for (i, ch) in enumerate(unique_ch)}
print(PAD_SYMBOL)

32


In [6]:
a = [ch for source in sources_ch for ch in source]
ch_encoder = OneHotEncoder()
ch_one_hot = ch_encoder.fit_transform(np.reshape(a, (-1, 1)))

In [7]:
ch_one_hot_sources = []
end_index = 0
for l in lengths_ch:
    ch_one_hot_sources.append(ch_one_hot[end_index:end_index+l])
    end_index += l

In [8]:
ch_one_hot_sources[0].toarray().shape

(355, 113)

In [14]:
one_hot_targets = np.eye(len(unique_ch))[32]
one_hot_targets

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [173]:
tf.reset_default_graph()
rnn_neurons_1 = 100
rnn_neurons_2 = 10
vec_dim = len(unique_ch)
BATCH_SIZE = 10

X = tf.placeholder(tf.float32, [None, None, vec_dim])
# X = tf.placeholder(tf.float32, (-1, vec_dim))
seq_length = tf.placeholder(tf.int32, [None])
# basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
rnn_layers = [tf.contrib.rnn.BasicRNNCell(size) for size in [rnn_neurons_1, rnn_neurons_2]]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
_, state = tf.nn.dynamic_rnn(cell=multi_layer_cell,
                             inputs=X,
                             dtype=tf.float32,
                             sequence_length = seq_length)

In [174]:
def get_batch(x_train, batch_size):
    x_batch = np.array([seq.toarray() for seq in ch_one_hot_sources[:batch_size]])
    seq_lengths = [seq.shape[0] for seq in x_batch]
    return x_batch, seq_lengths

def add_padding(batch, pad_size, padding):
    result = np.empty((0, pad_size, batch[0].shape[1]), np.float32)
    for i in range(len(batch)):
        seq = batch[i]
        if len(seq) < pad_size:
            seq = np.concatenate([seq, [padding for _ in range(pad_size - len(seq))]])
        result = np.append(result, [seq], axis=0)
    return result

In [175]:
init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

x_batch, batch_seq_len = get_batch(ch_one_hot_sources, BATCH_SIZE)
print(batch_seq_len)
# print(x_batch[1].shape)
x_batch = add_padding(x_batch, max(batch_seq_len), np.eye(vec_dim)[PAD_SYMBOL])
print(x_batch.shape)
# train_x = np.array(ch_one_hot_sources[:2])
# dim = train_x[0].toarray()
# print(dim)
# print(train_x)

sess.run(state, feed_dict={X: x_batch, seq_length: batch_seq_len})

[355, 327, 365, 338, 383, 355, 392, 365, 389, 361]
(10, 392, 113)


(array([[ 0.03925939,  0.11898942,  0.27921078, -0.3621077 ,  0.07536438,
          0.22532742, -0.00167355, -0.15026097, -0.10341782, -0.12017563,
         -0.12818114,  0.14979681, -0.210072  , -0.04952753, -0.09056833,
         -0.02350719,  0.01399339,  0.10335824,  0.24163495, -0.06882951,
          0.1244315 ,  0.00095457, -0.01130822,  0.21265139, -0.02042407,
         -0.07781149, -0.00222064, -0.03177413, -0.02103564, -0.25602412,
          0.13742644,  0.06876665, -0.18342036,  0.17084296, -0.21894373,
          0.0477253 ,  0.15432787, -0.10321715, -0.11154053,  0.04489494,
         -0.01780263, -0.15743518,  0.12171744, -0.0695234 , -0.00562614,
         -0.16535446,  0.01882941,  0.11154483,  0.10527736,  0.11481331,
          0.03958803, -0.12707794,  0.24788558,  0.0587776 ,  0.02295236,
          0.02274579, -0.0339086 ,  0.11422604,  0.05527326, -0.16891102,
          0.1361534 ,  0.25135309, -0.00520141,  0.02766245,  0.15977223,
         -0.20050025,  0.05561716,  0.